In [1]:
import numpy as np

def mod_inverse(matrix, modulus):
    det = int(np.round(np.linalg.det(matrix)))
    det_mod_inverse = pow(det, -1, modulus)
    adjugate_matrix = np.round(det * np.linalg.inv(matrix)).astype(int) % modulus
    return (det_mod_inverse * adjugate_matrix) % modulus

def encrypt_column_major(plaintext, key):
    n = len(key)
    plaintext = plaintext.upper().replace(" ", "")
    while len(plaintext) % n != 0:
        plaintext += "X"

    plaintext_numbers = [ord(char) - ord('A') for char in plaintext]

    # Fill plaintext matrix column by column
    plaintext_matrix = np.zeros((n, len(plaintext) // n), dtype=int)
    for col in range(len(plaintext) // n):
        for row in range(n):
            plaintext_matrix[row, col] = plaintext_numbers[col * n + row]

    ciphertext_matrix = (np.dot(key, plaintext_matrix) % 26).astype(int)

    # Flatten column by column
    ciphertext = ''.join(chr(ciphertext_matrix[row, col] + ord('A'))
                         for col in range(ciphertext_matrix.shape[1])
                         for row in range(n))
    return ciphertext

def decrypt_column_major(ciphertext, key):
    n = len(key)
    ciphertext = ciphertext.upper().replace(" ", "")
    ciphertext_numbers = [ord(char) - ord('A') for char in ciphertext]

    # Fill ciphertext matrix column by column
    ciphertext_matrix = np.zeros((n, len(ciphertext) // n), dtype=int)
    for col in range(len(ciphertext) // n):
        for row in range(n):
            ciphertext_matrix[row, col] = ciphertext_numbers[col * n + row]

    key_inverse = mod_inverse(key, 26)
    plaintext_matrix = (np.dot(key_inverse, ciphertext_matrix) % 26).astype(int)

    # Flatten column by column
    plaintext = ''.join(chr(plaintext_matrix[row, col] + ord('A'))
                        for col in range(plaintext_matrix.shape[1])
                        for row in range(n))
    return plaintext

if __name__ == "__main__":
    key_size = int(input("Enter the size of the key matrix (e.g., 2 for 2x2, 3 for 3x3): "))
    print(f"Enter the {key_size}x{key_size} key matrix (column by column) as characters:")
    key = []
    for _ in range(key_size):
        column = input().upper().replace(" ", "")
        key.append([ord(char) - ord('A') for char in column])
    key = np.array(key).T  # Transpose to follow column-major order

    plaintext = input("Enter the plaintext: ")
    ciphertext = encrypt_column_major(plaintext, key)
    print("Encrypted:", ciphertext)

    decrypted_text = decrypt_column_major(ciphertext, key)
    print("Decrypted:", decrypted_text)



Enter the size of the key matrix (e.g., 2 for 2x2, 3 for 3x3):  2


Enter the 2x2 key matrix (column by column) as characters:


 A B 
 B C
Enter the plaintext:  ATTACK


Encrypted: TMATKW
Decrypted: ATTACK
